### Motivation
* This notebook is to create a trusted San Diego County - Zip code feature layer
We will try to see if we can use the existing feature layer created from California Department of Education:  
Feature Layer: https://services3.arcgis.com/fdvHcZVgB2QSRNkL/arcgis/rest/services/ZipCodes/FeatureServer

`Description`: Five digit ZIP code areas used by the U.S. Postal Service in the state of California.


`Feature Layer Details`: https://gis.data.ca.gov/datasets/CDEGIS::california-zip-codes/explore?location=36.882750%2C-116.753799%2C6.65

In [12]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.geoenrichment import enrich, Country
from feature_layer_utils import get_enrichment_variables
import itertools
import sys
sys.path.append('../')
from utils import get_config
from gis_resources import san_diego_county_zips

In [3]:
app_id = get_config("arcgis","clientid")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=Ku1tKyuFo6t7WqfcHpshvHgeK6JGvM&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
fls = "https://services3.arcgis.com/fdvHcZVgB2QSRNkL/arcgis/rest/services/ZipCodes/FeatureServer"

In [6]:
flc = FeatureLayerCollection(fls)

In [7]:
fl_url = flc.layers[0].url

In [8]:
ca_zip_cd_fl = FeatureLayer(fl_url)

In [9]:
ca_zip_cd_fs = ca_zip_cd_fl.query()

In [10]:
ca_zip_cd_fs_sdf = ca_zip_cd_fs.sdf
print(f"Shape: {ca_zip_cd_fs_sdf.shape}")
ca_zip_cd_fs_sdf.head(4)

Shape: (1721, 10)


,OBJECTID,ZIP_CODE,PO_NAME,STATE,POPULATION,POP_SQMI,SQMI,Shape__Area,Shape__Length,SHAPE
0,1,00012,Mt Meadows Area,CA,-99,-99.0,30.92,862157443.22168,195388.61918,"{""rings"": [[[-235397.855560184, 9938636.425874..."
1,2,00016,Sequoia National Forest,CA,-99,-99.0,39.33,1096295677.53442,169790.572348,"{""rings"": [[[463415.329815902, 8230735.1641904..."
2,3,00017,Northeast Fresno County,CA,-99,-99.0,564.38,15734145627.6488,873109.20835,"{""rings"": [[[354976.177600153, 8923823.7678399..."
3,4,00018,Los Padres Ntl Forest,CA,-99,-99.0,90.83,2532221635.86206,294311.333366,"{""rings"": [[[355769.014925234, 7866496.3992311..."


In [11]:
#Dropping unwanted columns
ca_zip_cd_fs_sdf = ca_zip_cd_fs_sdf.drop(['POPULATION','POP_SQMI'], axis=1)
print(f"Shape: {ca_zip_cd_fs_sdf.shape}")
ca_zip_cd_fs_sdf.head(4)

Shape: (1721, 8)


,OBJECTID,ZIP_CODE,PO_NAME,STATE,SQMI,Shape__Area,Shape__Length,SHAPE
0,1,00012,Mt Meadows Area,CA,30.92,862157443.22168,195388.61918,"{""rings"": [[[-235397.855560184, 9938636.425874..."
1,2,00016,Sequoia National Forest,CA,39.33,1096295677.53442,169790.572348,"{""rings"": [[[463415.329815902, 8230735.1641904..."
2,3,00017,Northeast Fresno County,CA,564.38,15734145627.6488,873109.20835,"{""rings"": [[[354976.177600153, 8923823.7678399..."
3,4,00018,Los Padres Ntl Forest,CA,90.83,2532221635.86206,294311.333366,"{""rings"": [[[355769.014925234, 7866496.3992311..."


In [15]:
san_diego_zips = [str(x) for x in san_diego_county_zips()]
san_diego_zips[:10]

['91910',
 '91911',
 '91913',
 '91914',
 '91915',
 '91916',
 '91917',
 '91931',
 '91932',
 '91934']

In [16]:
san_diego_zip_sdf = ca_zip_cd_fs_sdf[ca_zip_cd_fs_sdf['ZIP_CODE'].isin(san_diego_zips)]
print(f"shape: {san_diego_zip_sdf.shape}")

shape: (109, 8)


In [17]:
san_diego_zip_sdf.head(3)

,OBJECTID,ZIP_CODE,PO_NAME,STATE,SQMI,Shape__Area,Shape__Length,SHAPE
340,341,91901,Alpine,CA,98.71,2751885514.80566,318537.985459,"{""rings"": [[[1043677.65911832, 7241310.2677161..."
341,342,91902,Bonita,CA,12.87,359079757.443848,149152.9387,"{""rings"": [[[935068.232308738, 7191189.7406608..."
342,343,91905,Boulevard,CA,130.91,3649534295.33887,384006.444396,"{""rings"": [[[1168374.38314957, 7195831.7710749..."


In [18]:
%%time
san_diego_zip_cd_fl = san_diego_zip_sdf.spatial.to_featurelayer(title='San Diego Zip Codes 1.0',
                                                        gis=gis,
                                                        folder='nourish_gis',
                                                        tags=['SanDiego','ZipCodes'],
                                                        service_name='San Diego Zip Codes 1.0'
                                                    )
san_diego_zip_cd_fl

CPU times: user 304 ms, sys: 94.1 ms, total: 398 ms
Wall time: 12.2 s


<Item title:"San Diego Zip Codes 1.0" type:Feature Layer Collection owner:akale_UCSDOnline>